<a href="https://colab.research.google.com/github/lily-880/Bear-classifier/blob/main/bear_classifier_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from fastai.vision.all import *
from pathlib import Path
import urllib.request


In [ ]:
# Cell 2 (fixed): force re-download from v2, show real URL/headers, then load learner
from fastai.vision.all import *
from pathlib import Path
import urllib.request
import traceback
import torch, fastai

MODEL_URL = "https://github.com/lily-880/Bear-classifier/releases/download/v2/export.pkl"
DOWNLOADED = Path("export.pkl")

print("fastai:", fastai.__version__)
print("torch :", torch.__version__)
print("MODEL_URL =", MODEL_URL)

try:
    # 1) Always re-download to avoid using an old cached/previous file
    if DOWNLOADED.exists():
        DOWNLOADED.unlink()

    print("Downloading model (forced)...")

    # 2) Follow redirects and log what we actually download
    req = urllib.request.Request(MODEL_URL, method="GET")
    with urllib.request.urlopen(req) as r:
        print("Final URL:", r.geturl())
        print("Status   :", getattr(r, "status", "N/A"))
        print("CT       :", r.headers.get("Content-Type"))
        print("CL       :", r.headers.get("Content-Length"))

        # Stream download to disk, but also keep first bytes for header check
        head = r.read(2048)
        with open(DOWNLOADED, "wb") as f:
            f.write(head)
            while True:
                chunk = r.read(1024 * 1024)
                if not chunk:
                    break
                f.write(chunk)

    print("Downloaded size:", DOWNLOADED.stat().st_size)
    header2 = DOWNLOADED.read_bytes()[:2]
    print("File header:", header2)

    # 3) Safety check: export.pkl from learn.export should NOT be a zip (PK)
    if header2 == b"PK":
        raise RuntimeError(
            "Downloaded file is a ZIP (header b'PK'), not a fastai export.pkl pickle. "
            "You likely uploaded the wrong file to the v2 release. "
            "Re-upload the correct ~47MB export.pkl produced by learn.export()."
        )

    # 4) Load learner
    print("Loading learner...")
    learn = load_learner(DOWNLOADED)
    print("Learner loaded OK.")

except Exception:
    print("=== Cell 2 failed: traceback below ===")
    traceback.print_exc()
    raise


In [ ]:
import ipywidgets as widgets
from IPython.display import display

uploader = widgets.FileUpload(accept='image/*', multiple=False)
out = widgets.Output()

def on_upload_change(change):
    out.clear_output()
    with out:
        img = PILImage.create(list(uploader.value.values())[0]['content'])
        pred, pred_idx, probs = learn.predict(img)
        display(img.to_thumb(256))
        print(f"Prediction: {pred}")
        print(f"Confidence: {probs[pred_idx]:.3f}")

uploader.observe(on_upload_change, names='value')

display(uploader, out)
